In [4]:
!pip install git+https://github.com/RobustBench/robustbench.git
!pip install -q foolbox
!pip install git+https://github.com/pralab/secml

import foolbox as fb
import torch
import torch.nn.functional as F
import secml
from robustbench.utils import load_model
from robustbench.data import load_cifar10

  Cloning https://github.com/RobustBench/robustbench.git to /tmp/pip-req-build-ifdbzcyt
  Running command git clone --filter=blob:none --quiet https://github.com/RobustBench/robustbench.git /tmp/pip-req-build-ifdbzcyt
  Resolved https://github.com/RobustBench/robustbench.git to commit 46a91f44524133b2cd8f721ec7e73ecb63f17fc8
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/fra31/auto-attack.git (to revision a39220048b3c9f2cca9a4d3a54604793c68eca7e) to /tmp/pip-install-t5bubr5l/autoattack_768af84aebb34063aaebdc3fee005dca
  Running command git clone --filter=blob:none --quiet https://github.com/fra31/auto-attack.git /tmp/pip-install-t5bubr5l/autoattack_768af84aebb34063aaebdc3fee005dca
  Running command git rev-parse -q --verify 'sha^a39220048b3c9f2cca9a4d3a54604793c68eca7e'
  Running command git fetch -q https://github.com/fra31/auto-attack.git a39220048b3c9f2cca9a4d3a54604793c68eca7e
  Resolved https://github.com/fra31/auto-attack.git to commit a39220048b3c9f2cca9a4

In [ ]:
def load_data():
    x_test, y_test = load_cifar10()
    return torch.tensor(x_test).float(), torch.tensor(y_test).long()

In [ ]:
model_names = [
    "Standard",
    "Rebuffi2021Fixing_70_16_cutmix_extra",
    "Gowal2020Uncovering_extra"
]

In [ ]:
def load_models(name):
    return load_model(model_name=name, threat_model='Linf')

In [1]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(),
                      lr=0.001, momentum=0.9)

model1 = load_models(model_names[0])
model2 = load_models(model_names[1])
model3 = load_models(model_names[2])

clf = CClassifierPyTorch(model= model1,
                         loss=criterion,
                         optimizer=optimizer,
                         input_shape=3,
                         random_state=999)

clf = CClassifierPyTorch(model= model2,
                         loss=criterion,
                         optimizer=optimizer,
                         input_shape=3,
                         random_state=999)

clf = CClassifierPyTorch(model= model3,
                         loss=criterion,
                         optimizer=optimizer,
                         input_shape=3,
                         random_state=999)

# fit the classifier
clf.fit(x_test, y_test)

# compute predictions on a test set
y_pred = clf.predict(x_test)

# Evaluate the accuracy of the classifier
metric = CMetricAccuracy()
acc = metric.performance_score(y_true=y_test, y_pred=y_pred)

print("Accuracy on test set: {:.2%}".format(acc))

In [ ]:
from secml.ml.classifiers.loss import CLossCrossEntropy
from secml.array import CArray


def pgd_l2_untargeted(x, y, clfs, eps, alpha, steps):
    for clf in clfs:
        clf.softmax_outputs = True

    loss_func = CLossCrossEntropy()
    x_adv = x.deepcopy()
    path = CArray.zeros((steps + 1, x.shape[1]))
    path[0, :] = x_adv

    for i in range(steps):
        combined_gradient = CArray.zeros(x.shape[1])

        for clf in clfs:
            scores = clf.decision_function(x_adv)
            loss_gradient = loss_func.dloss(y_true=y, score=scores)
            clf_gradient = clf.grad_f_x(x_adv, y)
            combined_gradient += clf_gradient * loss_gradient

        if combined_gradient.norm() != 0:
            combined_gradient /= combined_gradient.norm()

        x_adv = x_adv + alpha * combined_gradient
        delta = x_adv - x
        if delta.norm() > eps:
            delta = delta / delta.norm() * eps
            x_adv = x + delta
        x_adv = x_adv.clip(0, 1)
        path[i + 1, :] = x_adv

    for clf in clfs:
        clf.softmax_outputs = False

    predictions = [clf.predict(x_adv) for clf in clfs]
    return x_adv, predictions, path

In [3]:
x_test, y_test = load_data()

models = load_models()

for name, model in models.items():
    robust_acc = attack(model, x_test[:1000], y_test[:1000])
    print(f"Model {name}: Robust accuracy under attack = {robust_acc:.1%}")

NameError: name 'load_data' is not defined